# Problem 84 
## [Monopoly odds](https://projecteuler.net/problem=84)

<p>In the game, <i>Monopoly</i>, the standard board is set up in the following way:</p>
<div class="center">
<img src="https://projecteuler.net/project/images/p084_monopoly_board.png" alt="p084_monopoly_board.png" />
</div>
<p>A player starts on the GO square and adds the scores on two 6-sided dice to determine the number of squares they advance in a clockwise direction. Without any further rules we would expect to visit each square with equal probability: 2.5%. However, landing on G2J (Go To Jail), CC (community chest), and CH (chance) changes this distribution.</p>
<p>In addition to G2J, and one card from each of CC and CH, that orders the player to go directly to jail, if a player rolls three consecutive doubles, they do not advance the result of their 3rd roll. Instead they proceed directly to jail.</p>
<p>At the beginning of the game, the CC and CH cards are shuffled. When a player lands on CC or CH they take a card from the top of the respective pile and, after following the instructions, it is returned to the bottom of the pile. There are sixteen cards in each pile, but for the purpose of this problem we are only concerned with cards that order a movement; any instruction not concerned with movement will be ignored and the player will remain on the CC/CH square.</p>
<ul><li>Community Chest (2/16 cards):
<ol><li>Advance to GO</li>
<li>Go to JAIL</li>
</ol></li>
<li>Chance (10/16 cards):
<ol><li>Advance to GO</li>
<li>Go to JAIL</li>
<li>Go to C1</li>
<li>Go to E3</li>
<li>Go to H2</li>
<li>Go to R1</li>
<li>Go to next R (railway company)</li>
<li>Go to next R</li>
<li>Go to next U (utility company)</li>
<li>Go back 3 squares.</li>
</ol></li>
</ul><p>The heart of this problem concerns the likelihood of visiting a particular square. That is, the probability of finishing at that square after a roll. For this reason it should be clear that, with the exception of G2J for which the probability of finishing on it is zero, the CH squares will have the lowest probabilities, as 5/8 request a movement to another square, and it is the final square that the player finishes at on each roll that we are interested in. We shall make no distinction between "Just Visiting" and being sent to JAIL, and we shall also ignore the rule about requiring a double to "get out of jail", assuming that they pay to get out on their next turn.</p>
<p>By starting at GO and numbering the squares sequentially from 00 to 39 we can concatenate these two-digit numbers to produce strings that correspond with sets of squares.</p>
<p>Statistically it can be shown that the three most popular squares, in order, are JAIL (6.24%) = Square 10, E3 (3.18%) = Square 24, and GO (3.09%) = Square 00. So these three most popular squares can be listed with the six-digit modal string: 102400.</p>
<p>If, instead of using two 6-sided dice, two 4-sided dice are used, find the six-digit modal string.</p>



In [1]:
from collections import defaultdict
import random

In [2]:
tiles = [
    ["GO", 0],
    ["A", 1],
    ["CC", 1],
    ["A", 2],
    ["T", 1],
    ["R", 1],
    ["B", 1],
    ["CH", 1],
    ["B", 2],
    ["B", 3],
    ["JAIL", 0],
    ["C",1],
    ["U",1],
    ["C",2],
    ["C",3],
    ["R",2],
    ["D", 1],
    ["CC", 2],
    ["D", 2],
    ["D", 3],
    ["FP", 0],
    ["E", 1],
    ["CH", 2],
    ["E", 2],
    ["E", 3],
    ["R", 3],
    ["F", 1],
    ["F", 1],
    ["U", 2],
    ["F", 3],
    ["G2J", 0],
    ["G", 1],
    ["CC", 3],
    ["G", 2],
    ["G", 3],
    ["R", 4],
    ["CH", 3],
    ["H", 1],
    ["T", 2],
    ["H", 2],
]

In [3]:
monopoly = {}
for i in range(40):
    if i < 10:
        monopoly["0" + str(i)] = tiles[i]
    else:
        monopoly[str(i)] = tiles[i]

In [4]:
def setDice(n):
    def foo():
        first = random.randint(1, n)
        second = random.randint(1, n)
        return [first, second]
    return foo

In [5]:
def convertPos(n):
    n = n % 40
    if n < 10:
        return "0" + str(n)
    else:
        return str(n)

In [6]:
ordinary_type = ["A", "B", "C", "D", "E", "F", "G", "H", "U", "R", "FP", "T", "GO"]
special_type = ["CC", "CH", "JAIL", "G2J"]

In [7]:
def check_doubles(diceRess):
    if len(diceRess) < 3:
        return False
    if diceRess[-3][0] == diceRess[-3][1] and diceRess[-2][0] == diceRess[-2][1] and diceRess[-1][0] == diceRess[-1][1]:
        return True
    return False

In [17]:
cc = [["GOTO", "00"], ["GOTO", "10"]] + [[]] * 14
ch = [["GOTO", "00"], ["GOTO", "10"], ["GOTO", "11"], ["GOTO", "24"], ["GOTO", "39"], ["GOTO", "05"], ["GOTO", "NextR"], ["GOTO", "NextR"], ["GOTO", "NextU"], ["GOBACK", 3]] + [[]] * 6
def simulate(dice_num, simul_num, cc, ch):
    counts = defaultdict(int)
    roll_dice = setDice(dice_num)
    # roll dice
    dice_ress = []
    curr_pos = 0
    for _ in range(simul_num):
        dice_res = roll_dice()
        dice_ress.append(dice_res)
        dice_ress = dice_ress[-3:]

        if check_doubles(dice_ress):
            # doubles 3 times, go to jail
            curr_pos = "10"
        else:
            curr_pos = convertPos(int(curr_pos) + sum(dice_res))
        # curr_pos = convertPos(int(curr_pos) + sum(dice_res))
        curr_tile = monopoly[curr_pos]
        tile_type = curr_tile[0]
        if tile_type == "CC":
            card = cc[0]
            if card != []:
                curr_pos = card[1]
            cc = cc[1:] + [cc[0]]
        elif tile_type == "CH":
            card = ch[0]
            ch = ch[1:] + [ch[0]]
            if card != []:
                if card[0] == "GOBACK":
                    curr_pos = convertPos(int(curr_pos) - 3)
                elif card[1] == "NextR":
                    if curr_pos == "07":
                        curr_pos = "15"
                    elif curr_pos == "22":
                        curr_pos = "25"
                    elif curr_pos == "36":
                        curr_pos = "05"
                    else:
                        raise ValueError("1")
                elif card[1] == "NextU":
                    if curr_pos == "07" or curr_pos == "36":
                        curr_pos = "12"
                    elif curr_pos == "22":
                        curr_pos = "28"
                    else:
                        raise ValueError("2")
                else:
                    curr_pos = card[1]
        elif tile_type == "G2J":
            curr_pos = "10"
        counts[curr_pos] += 1

    for k in counts.keys():
        counts[k] = counts[k] / (simul_num) * 100
    return counts


In [25]:
%%timeit -n1 -r1
res = simulate(4, 10000000, cc, ch)
for k, v in sorted(res.items(), key=lambda s: s[1], reverse=True)[:10]:
    print(monopoly[k], k, v)

['JAIL', 0] 10 7.38478
['R', 2] 15 3.67518
['E', 3] 24 3.2699899999999995
['D', 1] 16 3.18466
['R', 3] 25 3.1169700000000002
['D', 3] 19 3.057
['E', 1] 21 3.03946
['FP', 0] 20 3.0089
['E', 2] 23 2.97756
['D', 2] 18 2.89779
25.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
